# COVID-19-FineTune-BERT-ResearchPapers-Semantic-Search

This work builds a **semantic search engine using BERT**, to search a query through the dataset of research papers provided as part of [Kaggle's competion CORD-19-research-challenge](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge), we like to thank kaggle and all of the competion sponsers for this competion in bringing up efforts for fighting this virus.

This work, 
1.   first divides the dataset to paragraphs
2.   FineTune BERT to the paragraphs
2.   then uses FineTuned BERT to embedded paragraphs of papers using 
3.   finally runs a query and returns the top 5 paragraphs and their papers' titles,abstract,abstract_summary

We have built this notebook to run seamlesly on google colab, connect with google drive, and downalod the data using [kaggle api](https://github.com/Kaggle/kaggle-api), so no data is downloaded to your device, and no need to have a powerful GPU, as all is done freely through google colab, we like to thank google for providing the research community with google colab

**Code** is found here [on github ](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search), we truly hope that this work has a postive impact in the fight aganist this evil virus, we truly pray for all people to be able to win this fight.


**References** :

*   We use the library provided by [huggingface](https://github.com/huggingface) called [transformers](https://github.com/huggingface/transformers), this library makes it truly easy to fineTune BERT 
*   We use the library provided by [UKPLab](https://github.com/UKPLab) called [sentence-transformers](https://github.com/UKPLab/sentence-transformers), this library makes it truly easy to use BERT and other architectures like ALBERT,XLNet for sentence embedding, they also provide simple interface to query and cluster data.
*   We have used the code from [maksimeren](https://www.kaggle.com/maksimeren/covid-19-literature-clustering) for data processing, we truly like to thank him.
*   We used the concept of drawing BERT, disccussed here [Jay Alammar](http://jalammar.github.io/) in illustrating how our architecture works, his blogs are extremly informative and easily understood.
*   We used the pre-trained models disccess in Conneau et al., 2017, show in the InferSent-Paper (Supervised Learning of Universal Sentence Representations from Natural Language Inference Data) that training on Natural Language Inference (NLI) data can produce universal sentence embeddings.
*  We have used vector elements from [freepik](freepik.com) for drawing





## Architecture

The research papers are found in the json from the dataset in paragraphs, we use this division in the papers, then we pass these paragraphs to finetune BERT


![alt text](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search/blob/master/assets/finetune/FineTune-04.jpg?raw=true)

Then we apply embedding 
![alt text](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search/blob/master/assets/finetune/Embedd%202-05.jpg?raw=true)

After embedding is done, we pass the embedded the query using the same bert model

Then we compare the both embedding represnetations (paragraphs and query) using cosine similarity, we then return the most similar paragraphs with their paper details (title,abstract,abstract_summary)

![alt text](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search/blob/master/assets/finetune/Cosine-06.jpg?raw=true)

## Install packages

https://mccormickml.com/2019/07/22/BERT-fine-tuning/

https://github.com/google-research/bert/blob/master/extract_features.py#L235

https://www.reddit.com/r/LanguageTechnology/comments/b3umhq/d_how_to_fine_tune_bert_embeddings_to_your_own/


In [ ]:
!pip install transformers

In [ ]:
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

In [ ]:
!git clone https://github.com/huggingface/transformers.git
#https://github.com/huggingface/transformers/issues/2693

## Download & Prepocess Data
or directly download preporcessed data from [here](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search#data-links) 

In [0]:
#---when running on google colab
##install the kaggle data to google colab
##https://github.com/Kaggle/kaggle-api#api-credentials
#!pip install kaggle
#import os
#!cp "/content/kaggle.json" /root/.kaggle
#!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge
#!unzip  CORD-19-research-challenge.zip -d /content/CORD-19-research-challenge

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

built using
https://www.kaggle.com/maksimeren/covid-19-literature-clustering

In [ ]:
import glob
import json
import pandas as pd
from tqdm import tqdm
root_path = '/kaggle/input/CORD-19-research-challenge/'
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

In [ ]:
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

### Read Data (Helpers)

In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            try:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            except:
                print("no abstract")
                self.abstract.append("")
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

In [ ]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [ ]:
from tqdm import tqdm 
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(tqdm(all_json)):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

### Handle Possible Duplicates

In [ ]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

In [ ]:
df_covid['body_text'].describe(include='all')

### Take a Look at the Data

In [ ]:
df_covid.head()

In [ ]:
df_covid.describe()

### Data Pre-Process

In [ ]:
df_covid.dropna(inplace=True)
df_covid.info()

In [ ]:
df_covid = df_covid.head(12500)

In [ ]:
import re

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [ ]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: lower_case(x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: lower_case(x))

In [ ]:
df_covid.head(4)

In [ ]:
df_covid.to_csv("covid.csv")

In [ ]:
df_covid_test = pd.read_csv("covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

In [ ]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])

In [ ]:
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

In [ ]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("covid_sentences.csv")
df_sentences.head()

In [ ]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("covid_sentences_Full.csv")
df_sentences.head()

## Preparing Data for Embedding

In [ ]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("covid_sentences.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [ ]:
df_sentences.head()

In [ ]:
import pandas as pd
df = pd.read_csv("covid_sentences_Full.csv", index_col=0)
df.head()

### Prepare input txt file

In [ ]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]
print(len(df_sentences_list))
df_sentences_list[:5]

In [ ]:
file_content = "\n".join(df_sentences_list)
with open("input_text.txt","w") as f:
  f.write(file_content)
#with open("input_text.txt","w") as f:
#  f.write(file_content[:-9000])
#with open("test_text.txt","w") as f:
#  f.write(file_content[-9000:])

In [0]:
print(len(file_content[:-9000]))
print(len(file_content[-9000:]))

## BERT

In [0]:
#export TRAIN_FILE=/path/to/dataset/wiki.train.raw
#export TEST_FILE=/path/to/dataset/wiki.test.raw
#https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/
#https://huggingface.co/transformers/v2.1.1/examples.html#roberta-bert-and-masked-language-modeling
!python "/content/transformers/examples/run_language_modeling.py" \
    --output_dir="/content/drive/My Drive/BertSentenceSimilarity/BERTfine" \
    --model_type=bert \
    --model_name_or_path=google/bert_uncased_L-2_H-128_A-2 \
    --do_train \
    --block_size=512 \
    --train_data_file="/content/input_text.txt" \
    --mlm

In [0]:
# if you want more RAMs from google colab
# fill your RAM using this simple loop
a = []
while(1):
    a.append('1')

In [0]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, losses
import scipy.spatial
import pickle as pkl
word_embedding_model = models.BERT("/content/drive/My Drive/BertSentenceSimilarity/BERTfine")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = model.encode(corpus,show_progress_bar=True)
with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_finetuned_embeddings.pkl" , "wb") as f:
  pkl.dump(corpus_embeddings,f)

In [0]:
len(corpus_embeddings)

In [0]:
# Query sentences:
queries = ['What has been published about medical care?',
           'Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest',
           'Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually',
           'Resources to support skilled nursing facilities and long term care facilities.',
           'Mobilization of surge medical staff to address shortages in overwhelmed communities .',
           'Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies .']
query_embeddings = model.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")

Results as run from google colab 



Batches: 100%|██████████| 1/1 [00:00<00:00, 116.54it/s]

Top 5 most similar sentences in corpus:


=========================================================
==========================Query==============================
=== What has been published about medical care? =====
=========================================================
Score:    (Score: 0.8779) 

Paragraph:    what is already known about this topic what are the new findings 

paper_id:   f084dcc7e442ab282deb97670e1843e347cf1fd5 

Title:    Ebola Holding Units at government hospitals in<br>Sierra Leone: evidence for a flexible and effective<br>model for safe isolation, early treatment<br>initiation, hospital safety and health system functioning 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8735) 

Paragraph:    to identify how one health has been used recently in the medical literature 

paper_id:   f703b510de361a759c9fe3419fa3aeb092c95512 

Title:    One Health and Zoonoses: The Evolution of One<br>Health and Incorporation of Zoonoses 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8727) 

Paragraph:    what is already known about the topic 

paper_id:   ea23ec453cc608e947baa6fe733bfbb400801f00 

Title:    Prevalence and correlates of influenza<br>vaccination among non-institutionalized elderly people:<br>An exploratory cross-sectional survey 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8691) 

Paragraph:    what is already known about this topic 

paper_id:   fac313563bd7a9ff7bb33495d81086ca75615951 

Title:    Implementing One Health as an integrated<br>approach to health in Rwanda 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8373) 

Paragraph:    3 summary statement about what is known and not known eg in the literature 

paper_id:   2079a44557da9aa2ce74d6af8ac0e8a344c1ad8d 

Title:    What Synthesis Methodology Should I Use? A<br>Review and Analysis of Approaches to Research<br>Synthesis 

Abstract:   background when we began this process we were doctoral students and a faculty member in a research methods course as students we were facing a review of the literature for our dissertations we encountered several different ways of conducting a review but were unable to locate any resources that synthesized all of the various synthesis methodologies our purpose is to present a comprehensive overview and assessment of the main approaches to research synthesis we use research synthesis as a broad overarching term to describe various approaches to combining integrating and synthesizing research findings methods we conducted an integrative review of the literature to explore the historical contextual and evolving nature of research synthesis we searched five databases reviewed websites of key organizations handsearched several journals and examined relevant texts from the reference lists of the documents we had already obtained results we identified four broad categories of research synthesis methodology including conventional quantitative qualitative and emerging syntheses each of the broad categories was compared to the others on the following key characteristics purpose method product context underlying assumptions unit of analysis strengths and limitations and when to use each approach conclusions the current state of research synthesis reflects significant advancements in emerging synthesis studies that integrate diverse data types and sources new approaches to research synthesis provide a much broader range of review alternatives available to health and social science students and researchers
research synthesis is the overarching term we use to describe approaches to combining aggregating integrating and synthesizing primary research findings each synthesis methodology draws on different types of findings depending on the purpose and product of the chosen synthesis see additional file 1
based on our current knowledge of the literature we identified these approaches to include in our review systematic review metaanalysis qualitative metasynthesis metanarrative synthesis scoping review rapid review realist synthesis concept analysis literature review and integrative review our first step was to divide the synthesis types among the research team each member did a preliminary search to identify key texts the team then met to develop search terms and a framework to guide the review
over the period of 2008 to 2012 we extensively searched the literature updating our search at several time points not restricting our search by date the dates of texts reviewed range from 1967 to 2015 we used the terms above combined with the term method eg realist synthesis and method in the database health source academic edition includes medline and cinahl this search yielded very few texts on some methodologies and many on others we realized that many documents on research synthesis had not been picked up in the search therefore we also searched google scholar pubmed eric and social science index as well as the websites of key organizations such as the joanna briggs institute the university of york centre for evidencebased nursing and the cochrane collaboration database we hand searched several nursing social science public health and health policy journals finally we traced relevant documents from the references in obtained texts
we included works that met the following inclusion criteria 1 published in english
2 discussed the history of research synthesis 3 explicitly described the approach and specific methods or 4 identified issues challenges strengths and limitations of the particular methodology we excluded research reports that resulted from the use of particular synthesis methodologies unless they also included criteria 2 3 or 4 above
based on our search we identified additional types of research synthesis eg metainterpretation best evidence synthesis critical interpretive synthesis metasummary grounded formal theory still we missed some important developments in metaanalysis for example identified by the journals reviewers that have now been discussed briefly in the paper the final set of 197 texts included in our review comprised theoretical empirical and conceptual papers books editorials and commentaries and policy documents
in our preliminary review of key texts the team inductively developed a framework of the important elements of each method for comparison in the next phase each text was read carefully and data for these elements were extracted into a table for comparison on the points of key characteristics purpose methods and product see additional file 1 once the data were grouped and extracted we synthesized across categories based on the following additional points of comparison complexity of the process degree of systematization consideration of context underlying assumptions unit of analysis and when to use each approach in our results we discuss our comparison of the various synthesis approaches on the elements above 

Abstract_Summary:    Background: When we began this process, we were<br>doctoral students and a faculty member in a research<br>methods course. As students, we were facing a review of<br>the literature for our dissertations. We<br>encountered several different ways of conducting a review<br>but were unable to locate any resources that<br>synthesized all of the various synthesis methodologies.<br>Our purpose is to present a comprehensive overview<br>and assessment of the main approaches to research<br>synthesis. We use 'research synthesis' as a broad<br>overarching term to describe various approaches to<br>combining, integrating, and synthesizing research<br>findings. Methods: We conducted an integrative review of... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest =====
=========================================================
Score:    (Score: 0.8942) 

Paragraph:    echocardiography and cardiac catheterization are common cardiac imaging modalities both modalities have drawbacks the limitations of echocardiography include operator dependence limited acoustic shadows a small field of view and poor evaluation of pulmonary veins the limitations of cardiac catheterization include the overlapping of great vessels difficulty in demonstrating systemic and pulmonary vessels at the same time catheterrelated complications and high doses of iodinated contrast material and ionizing radiation computed tomography ct can overcome these limitations 1 and previous studies widely reported its use in the evaluation of pediatric congenital heart disease 2e5 however there are longterm risks of developing lethal malignancy induced by radiation exposure of ct scan 6 7 children are at higher risk than adults because their tissues are more radiosensitive and they have longer life expectancies to develop a radiationinduced malignancy 8 recent studies have reported ct utilization trends in pediatric patients 9e11 fahimi et al 9 reported a dramatic increase in ct utilization among children presenting to the emergency department with abdominal pain menoch et al 10 showed a decrease in ct utilization in recent years in the pediatric emergency department of a regional pediatric referral center devries et al 11 evaluated ct utilization patterns in children with recurrent headache and reported overuse of head ct despite existing guidelines however limited data are available to describe contemporary trends in the utilization of pediatric cardiac ct and catheterizations the aim of this investigation is to study the impact of cardiac ct on the utilization of cardiac catheterization among children with congenital heart diseases in a tertiary referral general hospital 

paper_id:   66ebbf141431d784fb5f8887c6e840777504b868 

Title:    Trends in the utilization of computed<br>tomography and cardiac catheterization among children<br>with congenital heart disease 

Abstract:   backgroundpurpose pediatric cardiac computed tomography ct is a noninvasive imaging modality used to clearly demonstrate the anatomical detail of congenital heart diseases we investigated the impact of cardiac ct on the utilization of cardiac catheterization among children with congenital heart disease methods the study sample consisted of 2648 cardiac ct and 3814 cardiac catheterization from 1999 to 2009 for congenital heart diseases diagnoses were categorized into 11 disease groups the numbers of examination according to the different modalities were compared using temporal trend analyses the estimated effective radiation doses msv of ct and catheterization were calculated and compared results the number of ct scans and interventional catheterizations had a slight annual increase of 12 and 27 respectively whereas that of diagnostic catheterization decreased by 62 per year disease groups fell into two categories according to utilization trend differences between ct and diagnostic catheterization the increased use of ct reduces the need for diagnostic catheterization in patients with atrioventricular connection disorder coronary arterial disorder great vessel disorder septal disorder tetralogy of fallot and ventriculoarterial connection disorder clinicians choose either catheterization or ct or both examinations depending on clinical conditions in patients with semilunar valvular disorder heterotaxy myocardial disorder pericardial disorder and pulmonary vein disorder the radiation dose of ct was lower than that of diagnostic cardiac catheterization in all age groups conclusion the use of noninvasive ct in children with selected heart conditions might reduce the use of diagnostic cardiac catheterization this may release time and facilities within the catheterization laboratory to meet the increasing demand for cardiac interventions 

Abstract_Summary:    Background/Purpose: Pediatric cardiac<br>computed tomography (CT) is a noninvasive imaging<br>modality used to clearly demonstrate the anatomical<br>detail of congenital heart diseases. We investigated<br>the impact of cardiac CT on the utilization of<br>cardiac catheterization among children with<br>congenital heart disease. Methods: The study sample<br>consisted of 2648 cardiac CT and 3814 cardiac<br>catheterization from 1999 to 2009 for congenital heart<br>diseases. Diagnoses were categorized into 11 disease<br>groups. The numbers of examination, according to the<br>different modalities, were compared using temporal<br>trend analyses. The estimated effective radiation<br>doses (mSv) of CT and catheterization were<br>calculated and compared. Results: The... 

-------------------------------------------
Score:    (Score: 0.8937) 

Paragraph:    classic physical examination features of dcm include soft heart sounds from reduced contractility or pleural effusion gallop rhythm with or without a systolic murmur hypokinetic arterial pulses dull left apical impulse and clinical signs of profound chf exceptional cases are seen prior to onset of chf 

paper_id:   1fdc1a513291d7613c95bbefae5b7ea0e811d5a6 

Title:    Chapter 150 Cardiomyopathy 

Abstract:   1527
1528 section 11  cardiopulmonary system this chapter will next describe the clinical features of feline cardiomyopathies and the therapy of related complications following this is a consideration of canine dcm and arrhythmogenic cardiomyopathy
overview and pathophysiology of feline hcm  feline idiopathic hcm is characterized by hypertrophy and thickening of the left ventricle unexplained by congenital heart disease systemic hypertension or an endocrinopathy
 the condition is genetic in a number of feline breeds including the maine coon cat persian cat and the ragdoll thus far one sarcomeric mutation has been identified 

Abstract_Summary:    1527
1528 Section 11 / Cardiopulmonary System<br>This chapter will next describe the clinical<br>features of feline cardiomyopathies and the therapy of<br>related complications. Following this is a<br>consideration of canine DCM and arrhythmogenic<br>cardiomyopathy.
Overview and Pathophysiology of Feline HCM • Feline<br>idiopathic HCM is characterized by hypertrophy and<br>thickening of the left ventricle unexplained by<br>congenital heart disease, systemic hypertension, or an<br>endocrinopathy.
• The condition is genetic in a number of feline<br>breeds, including the Maine coon cat, Persian cat, and<br>the Ragdoll. Thus far, one sarcomeric mutation has<br>been identified. 

-------------------------------------------
Score:    (Score: 0.8905) 

Paragraph:    the clinical and imaging response to corticosteroids is rapid clinical manifestations abate within 48 hours and chest radiographs show a complete resolution of the pulmonary opacities within a few weeks without leaving significant functional or imaging sequelae although the efficiency of corticosteroids has been recognized for almost 20 years the ideal dose and duration of treatment have not been established however which underlines once again the difficulty of therapeutic trials in rare socalled orphan lung diseases 

paper_id:   0e4b438e603cfc47173173c2e131af30e8d99fe1 

Title:    Cryptogenic organizing pneumonia 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8887) 

Paragraph:    with over 11000 evd survivors there is increased recognition of a postebola syndrome in the convalescent period characterized by mental health and cognitive sequelae chronic headaches insomnia arthralgias auditory disturbances and ocular effects including sightthreatening uveitis [127] [128] [129] [130] [131] [132]  it is uncertain whether these manifestations are due to direct viral cytopathic effect in immuneprivileged compartments or postinfectious immunemediated inflammation [133] [134] [135] 

paper_id:   a52fd1386e8a463e46bde0e66125bfd826a64f56 

Title:    Ebola virus disease and critical illness 

Abstract:   as of 20 may 2016 there have been 28646 cases and 11323 deaths resulting from the west african ebola virus disease evd outbreak reported to the world health organization there continue to be sporadic flareups of evd cases in west africa evd presentation is nonspecific and characterized initially by onset of fatigue myalgias arthralgias headache and fever this is followed several days later by anorexia nausea vomiting diarrhea and abdominal pain anorexia and gastrointestinal losses lead to dehydration electrolyte abnormalities and metabolic acidosis and in some patients acute kidney injury hypoxia and ventilation failure occurs most often with severe illness and may be exacerbated by substantial fluid requirements for intravascular volume repletion and some degree of systemic capillary leak although minor bleeding manifestations are common hypovolemic and septic shock complicated by multisystem organ dysfunction appear the most frequent causes of death males and females have been equally affected with children 014 years of age accounting for 19  young adults 1544 years 58  and older adults 45 years 23  of reported cases while the current case fatality proportion in west africa is approximately 40  it has varied substantially over time highest near the outbreak onset according to available resources 4090  mortality in west africa compared to under 20  in western europe and the usa by age near universal among neonates and high among older adults and by ebola viral load at admission while there is no ebola virusspecific therapy proven to be effective in clinical trials mortality has been dramatically lower among evd patients managed with supportive intensive care in highly resourced settings allowing for the avoidance of hypovolemia correction of electrolyte and metabolic abnormalities and the provision of oxygen ventilation vasopressors and dialysis when indicated this experience emphasizes that in addition to evaluating specific medical treatments improving the global capacity to provide supportive critical care to patients with evd may be the greatest opportunity to improve patient outcomes 

Abstract_Summary:    As of 20 May 2016 there have been 28,646 cases and<br>11,323 deaths resulting from the West African Ebola<br>virus disease (EVD) outbreak reported to the World<br>Health Organization. There continue to be sporadic<br>flare-ups of EVD cases in West Africa. EVD presentation is<br>nonspecific and characterized initially by onset of<br>fatigue, myalgias, arthralgias, headache, and fever;<br>this is followed several days later by anorexia,<br>nausea, vomiting, diarrhea, and abdominal pain.<br>Anorexia and gastrointestinal losses lead to<br>dehydration, electrolyte abnormalities, and metabolic<br>acidosis, and, in some patients, acute kidney injury.<br>Hypoxia and ventilation failure occurs most often with<br>severe illness... 

-------------------------------------------
Score:    (Score: 0.8868) 

Paragraph:    a network of neurons that includes the ventrolateral medulla raphe nuclei and pons is responsible for controlling the duration of the inspiratory compressive and expulsive phases of coughing as well as the magnitude of motor activation of the respiratory muscles during this behavior 98  99  101  102  107 109 th is network also regulates respiratory muscles to produce breathing 102 evidence supporting the concept of an anatomically distributed but shared network for both cough and breathing comes from a variety of preclinical studies showing that brainstem neurons in these locations alter their activities during both behaviors 98  99  101  102  108 110 th ese neurons are synaptically connected in ways that can explain motor drive to respiratory muscles during both behaviors 98  99  101 th e process by which selected populations of this distributed brainstem network alter their activities to account for both behaviors is known as reconfi guration 98  101  102 although complex the reconfi gured brainstem network hypothesis for cough does not yet account for the diff erential regulation of this behavior relative to breathing furthermore this network cannot account for the phenotype of response to coughsuppressive drugs fi rst shown by may and widdicombe 111  50 years ago  centrally active antitussive drugs suppress cough in a dosedependent manner at doses that have no eff ect on breathing 111  112 by defi nition there must be some neural module in this brainstem network that is not required for breathing but is important for the full expression of cough some investigators have approached this problem using microinjection methods to apply antitussive drugs andor neurotransmitter agonists and antagonists to various brainstem regions 113 117 it is clear that familiar brainstem areas such as the nts could participate in the actions of antitussive drugs it is also clear that other areas not currently believed to have a command function for cough could be substrates for cough suppressants 113  115 these studies support the concept that there may be no single anatomic brainstem area or center that is susceptible to antitussive drugs but that these compounds may act at multiple sites when administered systemically th e extent to which these sites may contain undescribed elements of the brainstem network for cough is unknown but these fi ndings do support the idea that even though our current conceptualization of the brainstem cough network is complex it probably represents an underestimate of its true complexity 

paper_id:   2b9371136a7a90a486eddcaa7506c2ddebdad9a5 

Title:    Anatomy and Neurophysiology of Cough CHEST<br>Guideline and Expert Panel Report 

Abstract:   bronchopulmonary cfi bers and a subset of mechanically sensitive acidsensitive myelinated sensory nerves play essential roles in regulating cough these vagal sensory nerves terminate primarily in the larynx trachea carina and large intrapulmonary bronchi other bronchopulmonary sensory nerves sensory nerves innervating other viscera as well as somatosensory nerves innervating the chest wall diaphragm and abdominal musculature regulate cough patterning and cough sensitivity the responsiveness and morphology of the airway vagal sensory nerve subtypes and the extrapulmonary sensory nerves that regulate coughing are described the brainstem and higher brain control systems that process this sensory information are complex but our current understanding of them is considerable and increasing the relevance of these neural systems to clinical phenomena such as urge to cough and psychologic methods for treatment of dystussia is high and modern imaging methods have revealed potential neural substrates for some features of cough in the human chest 2014 146  6  1633 1648 abbreviations  atp 5 adenosine triphosphate  gerd 5 gastroesophageal reflux disease  git 5 gastrointestinal tract  nts 5 nucleus tractus solitarius  rar 5 rapidly adapting receptor  sar 5 slowly adapting receptor  trpa1 5 transient receptor potential a1  trpm8 5 transient receptor potential m8  trpv1 5 transient receptor potential vanilloid 1 [ evidencebased medicine ] 

Abstract_Summary:    Bronchopulmonary C-fi bers and a subset of<br>mechanically sensitive, acid-sensitive myelinated<br>sensory nerves play essential roles in regulating<br>cough. These vagal sensory nerves terminate<br>primarily in the larynx, trachea, carina, and large<br>intrapulmonary bronchi. Other bronchopulmonary sensory<br>nerves, sensory nerves innervating other viscera, as<br>well as somatosensory nerves innervating the chest<br>wall, diaphragm, and abdominal musculature<br>regulate cough patterning and cough sensitivity. The<br>responsiveness and morphology of the airway vagal sensory<br>nerve subtypes and the extrapulmonary sensory<br>nerves that regulate coughing are described. The<br>brainstem and higher brain control systems that process<br>this sensory information are complex, but our<br>current... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually =====
=========================================================
Score:    (Score: 0.9221) 

Paragraph:    another interpretation of our results is that while multiplex testing does well identifying influenza and other uricausing viruses in the ed implementation alone was not sufficient to significantly influence clinician decisionmaking and patient outcomes for example influenza testing alone may be enough for clinical decisionmaking during influenza season [9]  it is likely that a thoughtful and comprehensive stewardship program around rapid diagnostics could be required to lead to meaningful changes in outcomes it is also possible that deploying rapid diagnostic tools would lead to more significant changes if applied to an area that has higher baseline antibiotic prescription rates such as an urgent care setting 

paper_id:   0c88aef176b70c6b11a51028124680237a6e5abd 

Title:    Rapid Multiplex Testing for Upper Respiratory<br>Pathogens in the Emergency Department: A Randomized<br>Controlled Trial 

Abstract:   background acute upper respiratory tract infections are a common cause of emergency department ed visits and often result in unnecessary antibiotic treatment
methods we conducted a randomized clinical trial to evaluate the impact of a rapid multipathogen respiratory panel rp test vs usual care control patients were eligible if they were 12 months old had symptoms of upper respiratory infection or influenzalike illness and were not on antibiotics the primary outcome was antibiotic prescription secondary outcomes included antiviral prescription disposition and length of stay clinicaltrialsgov nct02957136
results of 191 patients enrolled 93 49 received rp testing 98 51 received usual care fiftythree 57 rp and 7 7 control patients had a virus detected and reported during the ed visit p  0001 twenty 22 rp patients and 33 34 usual care patients received antibiotics during the ed visit 12 95 confidence interval 25 to 04 p  06008 9 rp patients received antibiotics despite having a virus detected the magnitude of antibiotic reduction was greater in children 19 vs adults 9 post hoc analysis there was no difference in antiviral use length of stay or disposition
conclusions rapid rp testing was associated with a trend toward decreased antibiotic use suggesting a potential benefit from more rapid viral tests in the ed future studies should determine if specific groups are more likely to benefit from testing and evaluate the relative cost and effectiveness of broad testing focused testing and a combined diagnostic and antimicrobial stewardship approach 

Abstract_Summary:    Background. Acute upper respiratory tract<br>infections are a common cause of emergency department (ED)<br>visits and often result in unnecessary antibiotic<br>treatment.
Methods. We conducted a randomized clinical trial to<br>evaluate the impact of a rapid, multipathogen<br>respiratory panel (RP) test vs usual care (control).<br>Patients were eligible if they were ≥12 months old, had<br>symptoms of upper respiratory infection or<br>influenzalike illness, and were not on antibiotics. The<br>primary outcome was antibiotic prescription;<br>secondary outcomes included antiviral prescription,<br>disposition, and length of stay (ClinicalTrials.gov#<br>NCT02957136).
Results. Of 191 patients enrolled, 93 (49%) received RP<br>testing; 98 (51%) received usual care. Fifty-three... 

-------------------------------------------
Score:    (Score: 0.9189) 

Paragraph:    vaccine failure to protect from disease is usually due to problems with either client education or compliance with good animal management practices it is important for clients to understand the proper timing and method of vaccine administration what to realistically expect for vaccine efficacy and the importance of minimizing immunosuppressive factors and exposure to high doses of infectious agents in vaccinated animals 

paper_id:   915aba4e05fd2d54b26d3ecbddd3a810c0e0e69f 

Title:    Mechanistic Bases for Adverse Vaccine<br>Reactions and Vaccine Failures 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.9186) 

Paragraph:    large pragmatic multicentre randomised trials with appropriate controls and adequate randomisation procedures evaluating the impact of emergency care inservice training on longterm outcomes health professional practices and patient outcomes are needed given the current uncertainty on how long shortterm benefits are retained particularly in settings in which they are used infrequently 

paper_id:   737650f6ab6664da7582430d4b45ff9dd2e76866 

Title:    In-service training for health professionals<br>to improve care of seriously ill newborns and<br>children in low-income countries 

Abstract:   inservice training for health professionals to improve care of seriously ill newborns and children in lowincome countries review 

Abstract_Summary:    In-service training for health professionals<br>to improve care of seriously ill newborns and<br>children in low-income countries (Review) 

-------------------------------------------
Score:    (Score: 0.9178) 

Paragraph:    more research is necessary to better identify the specific organizational interventions needed to improve working conditions for healthcare professionals implementation research is particularly warranted to evaluate the effectiveness of various approaches in resourcelimited settings 

paper_id:   d4c836458692801c907aaca8ec7fc1c52b41df43 

Title:    Effect of effort‐reward imbalance and burnout<br>on infection control among Ecuadorian nurses 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.9170) 

Paragraph:    characterization of microbe transfer during realtime ems calls provides information on how to prioritize best practices for environmental decontamination in ems settings further development enforcement and evaluation of an infection control program with greater attention to decontamination of vehicle surfaces will result in reduced infectious microbe exposures for both ems responders and patients 

paper_id:   cb8b64a9b132b17468dc87a44eb83660b168f107 

Title:    Spread of infectious microbes during<br>emergency medical response 

Abstract:   background to our knowledge no studies to date demonstrate potential spread of microbes during actual emergency medical service ems activities our study introduces a novel approach to identification of contributors to ems environment contamination and development of infection control strategies using a bacteriophage surrogate for pathogenic organisms
methods bacteriophage fx174 was used to trace crosscontamination and evaluate current disinfection practices and a hydrogen peroxide h 2 o 2  wipe intervention within emergency response vehicles prior to ems calls 2 surfaces were seeded with fx174 on call completion ems vehicle and equipment surfaces were sampled before decontamination after decontamination per current practices and after implementation of the intervention results current decontamination practices did not significantly reduce viral loads on surfaces p  3113 but h 2 o 2 wipe intervention did p  0065 bacteriophage spread to 56 2748 of sites and was reduced to 54 2648 and 40 1948 with current decontamination practices and intervention practices respectively conclusion results suggest firefighters hands were the main vehicles of microbial transfer current practices were not consistently applied or standardized and minimally reduced prevalence and quantity of microbial contamination on ems surfaces although use of a consistent protocol of h 2 o 2 wipes significantly reduced percent prevalence and concentration of viruses training and promotion of surface disinfection should be provided 

Abstract_Summary:    Background: To our knowledge, no studies to<br>date demonstrate potential spread of microbes<br>during actual emergency medical service (EMS)<br>activities. Our study introduces a novel approach to<br>identification of contributors to EMS environment<br>contamination and development of infection control<br>strategies, using a bacteriophage surrogate for<br>pathogenic organisms.
Methods: Bacteriophage FX174 was<br>used to trace cross-contamination and evaluate<br>current disinfection practices and a hydrogen<br>peroxide (H 2 O 2 ) wipe intervention within emergency<br>response vehicles. Prior to EMS calls, 2 surfaces were<br>seeded with FX174. On call completion, EMS vehicle and<br>equipment surfaces were sampled before<br>decontamination, after decontamination per current... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Resources to support skilled nursing facilities and long term care facilities. =====
=========================================================
Score:    (Score: 0.8998) 

Paragraph:    three categories of human resources were identified as targets for ipc training and requiring different strategies and training contents ipc specialists all health care workers involved in service delivery and patient care and other personnel that support health service delivery administrative and managerial staff auxiliary service staff cleaners etc 

paper_id:   7cd8d3fec1f3a8251b75a50aad856e1b5526de83 

Title:    Core components for effective infection<br>prevention and control programmes: new WHO<br>evidence-based recommendations 

Abstract:   health careassociated infections hai are a major public health problem with a significant impact on morbidity mortality and quality of life they represent also an important economic burden to health systems worldwide however a large proportion of hai are preventable through effective infection prevention and control ipc measures improvements in ipc at the national and facility level are critical for the successful containment of antimicrobial resistance and the prevention of hai including outbreaks of highly transmissible diseases through high quality care within the context of universal health coverage given the limited availability of ipc evidencebased guidance and standards the world health organization who decided to prioritize the development of global recommendations on the core components of effective ipc programmes both at the national and acute health care facility level based on systematic literature reviews and expert consensus the aim of the guideline development process was to identify the evidence and evaluate its quality consider patient values and preferences resource implications and the feasibility and acceptability of the recommendations as a result 11 recommendations and three good practice statements are presented here including a summary of the supporting evidence and form the substance of a new who ipc guideline 

Abstract_Summary:    Health care-associated infections (HAI) are a<br>major public health problem with a significant<br>impact on morbidity, mortality and quality of life.<br>They represent also an important economic burden to<br>health systems worldwide. However, a large<br>proportion of HAI are preventable through effective<br>infection prevention and control (IPC) measures.<br>Improvements in IPC at the national and facility level are<br>critical for the successful containment of<br>antimicrobial resistance and the prevention of HAI,<br>including outbreaks of highly transmissible diseases<br>through high quality care within the context of<br>universal health coverage. Given the limited<br>availability of IPC evidencebased guidance and standards,<br>the World... 

-------------------------------------------
Score:    (Score: 0.8947) 

Paragraph:    kslp with the support of other organisations such as the international nongovernmental organisation goal for construction worked with local colleagues to design and construct the facility train local staff on safety and patient care develop local clinical pathways oversee clinical care and troubleshoot problems or difficult cases develop administrative systems such as stock systems and patient records provide additional manpower  particularly at the physician level and rolemodelling and provide additional medical supplies where items were not available or in short supply through the government supply chain 

paper_id:   f084dcc7e442ab282deb97670e1843e347cf1fd5 

Title:    Ebola Holding Units at government hospitals in<br>Sierra Leone: evidence for a flexible and effective<br>model for safe isolation, early treatment<br>initiation, hospital safety and health system functioning 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8947) 

Paragraph:    raising awareness and providing training in the use of nlms disaster health resources and providing training on their use 

paper_id:   7aa69b98902425b5daeb3b1f647d067b8abf7157 

Title:    National Library of Medicine Disaster<br>Information Management Research Center: Achieving the<br>vision, 2010–2013 

Abstract:   from 2010 to 2013 the national library of medicine nlm disaster information management research center dimrc continued to build its programs and services on the foundation laid in its starting years 20082010 prior to 2008 nlm had a long history of providing health information training and tools in response to disasters aware of this legacy the nlm long range plan charting a course for the 21st century nlms long range plan 20062016 called for creation of a center to show a strong commitment to disaster remediation and to provide a platform for demonstrating how libraries and librarians can be part of the solution to this national problem nlm is continuing efforts to ensure that medical libraries have plans for the continuity of their operations librarians are trained to understand their roles in preparedness and response online disaster health information resources are available for many audiences and in multiple formats and research is conducted on tools to enhance the exchange of critical information during and following disasters this paper describes the 20102013 goals and activities of dimrc and its future plans
keywords
ensuring that tools and resources are available in a variety of formats using the latest technology commonly in use
outreach and partnerships collaborating with other federal agencies state and local communities public health officials and emergency managers in efforts to prevent respond to and reduce the adverse health effects of disasters
partnering with other government agencies involved in disaster health to ensure information needs receive adequate attention in planning for disasters and are included in training for responders
providing guides to disaster medicine and public health information resources for specific incidents such as the haiti earthquake and cholera outbreak the gulf of mexico oil spill the earthquake tsunami and radiation incident in japan and hurricane sandy
collaborating with nlm its eight regional medical libraries and 6000 memberlibraries of its national network of libraries of medicine nnlm [31] in planning and training for continuity of operations for library facilities services and staff
this paper explores each of these areas of interest and also includes guidance provided by an advisory group established by the nlm board of regents and by the dimrc strategic plan 2012201320142015
love et al 

Abstract_Summary:    From 2010 to 2013, the National Library of<br>Medicine (NLM) Disaster Information Management<br>Research Center (DIMRC) continued to build its programs<br>and services on the foundation laid in its starting<br>years, 2008-2010. Prior to 2008, NLM had a long history<br>of providing health information, training, and<br>tools in response to disasters. Aware of this legacy,<br>the NLM long range plan (Charting a Course for the<br>21st Century: NLM's Long Range Plan 2006-2016)<br>called for creation of a center to show "a strong<br>commitment to disaster remediation and to provide a<br>platform for demonstrating how libraries and<br>librarians can be part... 

-------------------------------------------
Score:    (Score: 0.8946) 

Paragraph:    arrangements with academic centers afford opportunities to improve laboratory services and surveillance systems flexible funding and other resources such as grants faculty and students help support special research projects and training opportunities within public health laboratories 

paper_id:   beb2c89f55d3ae239daf4b66fa58b8976351f064 

Title:    CHAPTER 8 Laboratories 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8863) 

Paragraph:    morbidity and mortality conferences mmcs are a traditional tool of improving local care management and clinical management education especially in high risk specialty but they lack a precise format for practice in intensive care units icu 

paper_id:   5a6330a739f18fd6bc502bd9b59de55e8c081d4e 

Title:    ESICM LIVES 2016: part one: Milan, Italy. 1-5<br>October 2016 

Abstract:   intensive care medicine experimental 2016 4suppl 1a1
introduction pharmacological immunomodulatory interventions in acute respiratory distress syndrome ards have been unsuccessful in clinical trials [13] despite promising results in preclinical studies using animals [4][5] poor phenotyping of patients could be responsible for these disappointing results objectives we hypothesized that ards patients can be clustered based on concentrations of plasma biomarkers and that such biological endophenotypes are association with clinical outcomes methods patients were screened for presence of ards unbiased cluster analysis of plasma concentrations of 20 biomarkers of inflammation coagulation and endothelial activation at diagnosis of ards provided the endophenotypes a decision tree was then used to predict cluster membership based on a more restricted set of biomarkers the independent association of endophenotypes with icu mortality was studied by multivariate logistic regression results three endophenotypes of ards were identified in 771 patients which we named impassive n  383 intermediate n  224 and reactive n  164 had mortality rates of 16  26  and 47  respectively p  001 patients with a reactive endophenotype were younger had higher disease severity scores more failing organs and more frequently had an indirect cause for ards than patients with an impassive or intermediate endophenotype a reactive endophenotype was independent from confounders associated with icu mortality or 118 [95  confidence interval 109128] the concentration of interleukin 10 interleukin 8 and matrix metalloproteinase 8 were sufficient to predict the three endophenotypes conclusions ards patients can be clustered into three biological endophenotypes with different mortality rates three easy to measure biomarkers can be used to predict the endophenotype 

Abstract_Summary:    Intensive Care Medicine Experimental 2016,<br>4(Suppl 1):A1
Introduction: Pharmacological<br>immunomodulatory interventions in 'acute respiratory distress<br>syndrome' (ARDS) have been unsuccessful in clinical<br>trials [1-3] despite promising results in<br>preclinical studies using animals [4][5]. Poor<br>phenotyping of patients could be responsible for these<br>disappointing results. Objectives: We hypothesized that<br>ARDS patients can be clustered based on<br>concentrations of plasma biomarkers and that such biological<br>endophenotypes are association with clinical outcomes.<br>Methods: Patients were screened for presence of ARDS.<br>Unbiased cluster analysis of plasma concentrations of<br>20 biomarkers of inflammation, coagulation and<br>endothelial activation at diagnosis of ARDS provided the<br>endophenotypes. A decision... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Mobilization of surge medical staff to address shortages in overwhelmed communities . =====
=========================================================
Score:    (Score: 0.8879) 

Paragraph:    rrt members epidemic intelligence service officers kcdc workers and staffs in community health centers attended onsite rrt through confirming with hospital officials we conducted a consultation rapidly about closing the hospitals isolation of patients and medical staffs and infection control in hospital 

paper_id:   954d000b2e7252ffb05812341a6436d3d784494a 

Title:    Collaborative Intervention of Middle East<br>Respiratory Syndrome: Rapid Response Team 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8837) 

Paragraph:    as discussed previously the number of responding hospitals evolved over time the challenges of identifying a role for local public health bureaus in the cdcmn using negative pressure isolation wards during nonemergency operations and operating within a decreasing cdcmn budget resulted in a change in the number of regional responding hospitals from 25 hospitals in 2012 to 22 hospitals in 2013 to 6 hospitals in 2014 this alteration enhanced the local public health bureaus role in the hospital preparedness efforts by allowing them to identify and coordinate local responding hospitals and enabled local hospitals to use negative pressure isolation rooms during daily operations which better justified the investment required to maintain them in addition the 6 regional responding hospitals received more cdcmn resources previously diverted to 25 hospitals to maximize the effectiveness the changes create a more costeffective network that still maintains flexibility taiwans hospital preparedness and response and surge capacity during an epidemic although changes from 2013 to date showed improvement in the operation and budget allocation current challenges such as the cost for renewal and replacement of the medical equipment in responding hospitals the development of integrated and comprehensive training courses of core skills for healthcare workers and risk compensation payments need the taiwanese governments ongoing commitment and collaboration with medical partners to continue efforts in enhancing a more solid and effective system 

paper_id:   cc6cb3af2ef6432cf20dd2c84b56fdb8be780078 

Title:    Taiwan's Experience in Hospital Preparedness<br>and Response for Emerging Infectious Diseases 

Abstract:   taiwan has undergone several phases of modification in structure and activation the main organizing principles of the cdcmn are centralized isolation of patients with severe highly infectious diseases and centralization of medical resources as well as a network of designated regional hospitals like those in other countries the cdcmn is made up of a command system responding hospitals and supporting hospitals it was tested and activated in response to the h1n1 influenza pandemic in 200910 and the ebola outbreak in west africa in 20142016 and it demonstrated highlevel functioning and robust capacity in this article the history structure and operation of the cdcmn is introduced globally for the first time and the advantages and challenges of this system are discussed the taiwanese experience shows an example of a collaboration between the public health system and the medical system that may help other public health authorities plan management and hospital preparedness for highly infectious diseases 

Abstract_Summary:    Taiwan, has undergone several phases of<br>modification in structure and activation. The main<br>organizing principles of the CDCMN are centralized<br>isolation of patients with severe highly infectious<br>diseases and centralization of medical resources, as<br>well as a network of designated regional hospitals<br>like those in other countries. The CDCMN is made up of<br>a command system, responding hospitals, and<br>supporting hospitals. It was tested and activated in<br>response to the H1N1 influenza pandemic in 2009-10 and<br>the Ebola outbreak in West Africa in 2014-2016, and<br>it demonstrated high-level functioning and<br>robust capacity. In this article, the history,<br>structure, and operation... 

-------------------------------------------
Score:    (Score: 0.8795) 

Paragraph:    the terrorist attacks of september 11 2001 and the anthrax attacks that followed in october propelled astho to its next level of growth  figure  amid wide recognition that governmental public health is critical to emergency preparedness and response large investments were made at the local and state levels in public health preparedness capacity nationwide astho received federal dollars through cooperative agreements with the cdc and the assistant secretary for preparedness and response to convene state and territorial public health officers and preparedness directors support state efforts to build preparedness and response programs and assist in national disasters and emergencies as a response partner the modern era of public health preparedness had begun as did a new normal in public health that required active involvement in homeland security efforts on a basis of 24 hours 7 days a week discussions around how federal dollars for preparedness should be allocated at the state and local levels led to the development of the asthonaccho national association of county  city health officials joint council convening health officers to discuss coordinated emergency response efforts to hurricane katrina the gulf of mexico oil spill the h1n1 outbreak and severe acute respiratory syndrome outbreak laid the groundwork for asthos role in supporting state and territorial health agencies and federal partners during public health disasters working collaboratively with naccho the asthonaccho joint council also became an increasingly important factor in policy formulation and advocacy for governmental public health 

paper_id:   c506acf42e11fa9de3408fd557799de21ec89080 

Title:    ASTHO at 75: Celebrating the Past and Preparing<br>for the Future 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8785) 

Paragraph:    similar to the findings of gershon et al [13] of distress and transition difficulties upon returning home this study found negative public sentiment towards returning health care workers influenced by media reports and confusing public health policies around quarantine created distress in participants this has the potential to reduce willingness to respond in future outbreaks clear and consistent messaging from both political and public health bodies may go a long way in tempering public panic and creating an atmosphere of support for health workers 

paper_id:   1eadeab29c774bb8b5935c5ce5d741855a8080a5 

Title:    The role of risk perception in willingness to<br>respond to the 2014–2016 West African Ebola outbreak: a<br>qualitative study of international health care workers. 

Abstract:   background the 20142016 west africa ebola virus disease evd outbreak was an unprecedented public health event and in addition to claiming over 11000 lives it resulted in the deaths of more healthcare workers than any outbreak in recent history while a cadre of willing and able health workers is essential for an effective epidemic response health workforce capacity in times of crisis may be significantly impacted by how risks are perceived by health staff this study aimed to explore how risk perceptions influenced healthcare workers willingness to respond during this outbreak
methods through indepth interviews with 11 frontline international health care workers who chose to respond to the west africa outbreak this qualitative study explores how perceptions of risk developed and subsequently mediated the decision to respond to the outbreak data was thematically organized using nvivo 10
results we found that numerous individual and sociallevel factors played a role in modifying risk perception in health workers institutional trust emerged as a key risk attenuator as did past experience selfefficacy duty of care humanitarian ethos and cognitive heuristics feelings of risk were amplified by infections of coworkers and risk perceptions of family members and the public which were mainly informed by media reports also hampered willingness to respond in some cases
conclusions understanding the risk perceptions of health workers institutions and the public while complex and interdependent are each crucial to understand for an effective public health response to epidemics and as such should be taken into consideration in future program planning and research 

Abstract_Summary:    Background: The 2014-2016 West Africa Ebola<br>Virus Disease (EVD) outbreak was an unprecedented<br>public health event, and in addition to claiming over<br>11,000 lives, it resulted in the deaths of more<br>healthcare workers than any outbreak in recent history.<br>While a cadre of willing and able health workers is<br>essential for an effective epidemic response, health<br>workforce capacity in times of crisis may be<br>significantly impacted by how risks are perceived by health<br>staff. This study aimed to explore how risk<br>perceptions influenced healthcare workers' willingness<br>to respond during this outbreak.
Methods:<br>Through in-depth interviews with 11 front-line<br>international health care... 

-------------------------------------------
Score:    (Score: 0.8777) 

Paragraph:    the mpss was formed by the end of 2014 before its establishment both fire stations and the local emergency management departments worked on disaster management in their local regions while firefighters focused on extinguishing fires civil engineers made efforts to deal with floods accompanied by typhoons similar to the case of the central government only two categories of professionals namely firefighters and civil engineers were allowed to work for the local governments 

paper_id:   51d4a11f19e49f08f97d11ab7b22ac934db1da74 

Title:    Disasters can happen to anybody: The case of<br>Korea 

Abstract:   in korea there is a pervasive feeling of invincibility to the point that people and organizations do not believe that disasters can strike them this has impact on the level of preparedness for disasters this study aims to delve into how korea has to change its governmental policiespractices with some private partners efforts to mitigate disaster risks a case study was utilized as the major methodology by comparing exclusive management with inclusive management these two approaches have been comparatively analyzed via four variables namely the central government the local governments the incident commander and other stakeholders the major finding is that koreas practices and policies have to evolve from the current exclusive management into futureoriented inclusive management moreover the importance of communication cooperation collaboration and multidiscipline coordination is discussed additionally the problem of reductionism and equal participation among all stakeholders as well as the resistance from vested interests are recognized and elaborated for korea and the international community 

Abstract_Summary:    In Korea, there is a pervasive feeling of<br>invincibility to the point that people and organizations do<br>not believe that disasters can strike them. This<br>has impact on the level of preparedness for<br>disasters. This study aims to delve into how Korea has to<br>change its governmental policies/practices with<br>some private partners' efforts to mitigate<br>disaster risks. A case study was utilized as the major<br>methodology by comparing exclusive management with<br>inclusive management. These two approaches have been<br>comparatively analyzed via four variables, namely the<br>central government, the local governments, the<br>incident commander, and other stakeholders. The major<br>finding is... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies . =====
=========================================================
Score:    (Score: 0.9133) 

Paragraph:    clinical behavior was assessed in terms of the main complications previously described for hmpv such as length of hospital stay need for transfer to intensive care need for ventilatory support and duration of mechanical ventilation acute respiratory infections were classified as rhinopharyngitis laryngitis croup bronchiolitis tracheitis pneumonia or acute respiratory distress syndrome ards according to the criteria of the attending physician 

paper_id:   aefd3e6661ed4befc6dd11d15cb80a66e025102e 

Title:    Prevalence, clinical outcomes and rainfall<br>association of acute respiratory infection by human<br>metapneumovirus in children in Bogotá, Colombia 

Abstract:   background acute respiratory infections aris are one of the main causes of morbidity and mortality in children viruses are the main etiological agents and their behavior tends to be seasonal and vary by geographical location human metapneumovirus hmpv has recently been described as a cause of severe acute respiratory infection and its prevalence and clinical behavior in children at moderate altitudes is unknown methods a crosssectional study was carried out on patients seen at a university hospital in bogot colombia between october 2015 and december 2017 in a city at a moderate altitude above sea level children with acute respiratory infections who had had a multiplex rtpcr assay were selected the prevalence of hmpv infection its clinical outcomes and its relationship to rainfall were evaluated results out of a total of 14760 discharged patients multiplex rtpcr was performed on 502 and a virus was detected in 420 children with acute respiratory infection ari the study group had a median age of 21 months iqr 760 with similar proportion of males and females 564 and 436 respectively and 52 ci 95 3378 prevalence of hmpv infection the group with hmpv infection showed a greater frequency of viral coinfection 227 vs 14 p  003 compared with ari caused by other viruses the rate of bacterial coinfection p  031 presence of comorbidities p  075 length of hospital stay p  042 need for mechanical ventilation p  075 and mortality p  022 were similar for hmpv and other viral infections a moderate correlation was established between hmpv infection and rainfall peaks spearmans rho 044 p  002 conclusions human metapneumovirus was the fifth most frequently isolated virus in children with ari had similar clinical behavior and severity to other viruses but a higher rate of viral coinfection its peaks seem to correlate to rainy seasons 

Abstract_Summary:    Background: Acute respiratory infections<br>(ARIs) are one of the main causes of morbidity and<br>mortality in children. Viruses are the main etiological<br>agents, and their behavior tends to be seasonal and vary<br>by geographical location. Human<br>metapneumovirus (HMPV) has recently been described as a cause of<br>severe acute respiratory infection and its<br>prevalence and clinical behavior in children at moderate<br>altitudes is unknown. Methods: A cross-sectional study<br>was carried out on patients seen at a university<br>hospital in Bogotá, Colombia between October 2015 and<br>December 2017 in a city at a moderate altitude above sea<br>level. Children with acute respiratory... 

-------------------------------------------
Score:    (Score: 0.9125) 

Paragraph:    to the editor total knee arthroplasty tka is one of the most common orthopedic surgeries to reduce pain and improve function for patients with osteoarthritis however tka is associated with highrisk of perioperative morbidity and mortality we presented a patient who developed severe acute respiratory distress syndrome ards postoperatively after bilateral tka procedure 

paper_id:   8e101b19cc15410540e85ce7f2e77208e3c411d5 

Title:    Severe Acute Respiratory Distress Syndrome<br>after Bilateral Total Knee Replacement 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.9101) 

Paragraph:    the most common physiologic abnormality is hypoxemia that may persist for weeks after apparent recovery 255 prolonged pulmonary function abnormalities including increased airway resistance peripheral airway obstruction and decreased arterial oxygen saturation have been detected in children years after bouts of bronchiolitis 256 bronchiolitis in infancy has also been associated with an increased risk of subsequent recurrent wheezing and cough and airway hyperreactivity 

paper_id:   6eb2300a2c10e3791188d4f355aa04a93c613b40 

Title:    32 Viral Infections 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.9088) 

Paragraph:    the following variables were collected from the patients medical histories according to international definitions age gender and comorbidities such as chronic obstructive pulmonary disease copd diabetes mellitus and heart failure the inclusion criteria took into account the systemic inflammatory response syndrome sirs which was defined as a heart rate over 90 beats per minute a breathing rate of over 20 breaths per minute leukocytosis of over 12000 cells per millilitre or leukopenia of fewer than 4000 cells per millilitre and fever pneumonia among sari patients was diagnosed from radiological findings of consolidation or alveolar infiltrates the confusion urea respiratory rate and blood pressure curb65 scores for pneumonia were applied using the criteria of age greater than or equal to 65 years impaired consciousness blood urea nitrogen bun over 20 mgdl breathing rate above 30 breaths per minute and a systolic arterial pressure under 90 mmhg or a diastolic pressure under 60 mmhg [8]  additionally severe pneumonia was considered among patients complying with the following major and minor criteria of the american thoracic societyinfectious disease society of america atsidsa shock need for mechanical ventilation thrombocytopenia platelets under 100000 an arterial oxygen partial pressure to fractional inspired oxygen ratio pao 2 fio 2  ratio multilobar compromise impaired consciousness leukopenia under 4000 and bun over 20 mgdl [9]  shock was also considered when the systolic arterial pressure was below 90 mmhg or when the diastolic arterial pressure was under 60 mmhg moderate oxygenation impairment was defined by a pao 2 fio 2 below 220 mmhg and over 160 mmhg and severe oxygenation impairment was defined by an index less than or equal to 160 mmhg institutions that identified viral antigens following institutional protocols performed and interpreted these tests in their hospitals colonisation of the airway was defined as the presence of microorganisms in a gram test or in a culture of the airway gram tests and sputum cultures or other respiratory samples were performed based on clinical decisions made by the doctor from the institution a viral coinfection diagnosis was made when 2 or more different viruses were identified in the same sample 

paper_id:   197679559a6aa8d3f30bde60d9387f1f9f975c0a 

Title:    Viral Infection in Adults with Severe Acute<br>Respiratory Infection in Colombia 

Abstract:   to identify the viral aetiology in adult patients with severe acute respiratory infection sari admitted to sentinel surveillance institutions in bogot in 2012
a crosssectional study was conducted in which microarray molecular techniques for viral identification were used on nasopharyngeal samples of adult patients submitted to the surveillance system and further descriptions of clinical features and relevant clinical outcomes such as mortality need for critical care use of mechanical ventilation and hospital stay were obtained
respiratory infections requiring hospital admission in surveillance centres in bogot colombia
ninetyone adult patients with acute respiratory infection 55 were female
viral identification intensive care unit admission hospital stay and mortality
viral identification was achieved for 63 patients 692 comorbidity was frequently identified and mainly involved chronic pulmonary disease or pregnancy influenza bocavirus and adenovirus were identified in 308 286 and 187 of the cases respectively admission to the intensive care unit occurred in 429 of the cases while mechanical ventilation was required for 363 the average hospital stay was 99 days and mortality was 154 antibiotics were empirically used in 901 of patients
the prevalence of viral aetiology of sari in this study was high with adverse clinical outcomes intensive care requirements and high mortality
this descriptive crosssectional study was performed in bogot in 2012 to determine the prevalence of viruses in adult patients with sari of less than 15 days of evolution using a microarray technique the study also describes clinical characteristics and outcomes such as mortality the virus in respiratory infections plos one  

Abstract_Summary:    To identify the viral aetiology in adult<br>patients with severe acute respiratory infection<br>(SARI) admitted to sentinel surveillance<br>institutions in Bogotá in 2012.
A cross-sectional study was<br>conducted in which microarray molecular techniques for<br>viral identification were used on nasopharyngeal<br>samples of adult patients submitted to the<br>surveillance system, and further descriptions of clinical<br>features and relevant clinical outcomes, such as<br>mortality, need for critical care, use of mechanical<br>ventilation and hospital stay, were<br>obtained.
Respiratory infections requiring hospital admission in<br>surveillance centres in Bogotá, Colombia.
Ninety-one<br>adult patients with acute respiratory infection<br>(55% were female).
Viral identification,<br>intensive care unit admission, hospital stay,... 

-------------------------------------------
Score:    (Score: 0.9087) 

Paragraph:    percentage of viruses is highly variable and was as high as 36 in one study and may be higher in infants and young children than in adults rales consolidation bronchial breath sounds pleural effusion dullness and decreased breath sounds or bronchial congestion rhonchi and wheezing findings consistent with pneumonia include fever tachypnea tachycardia decreased oxygen saturation and altered mental status associated with severe illness 

paper_id:   9b2a212c653cdc3405e174e9865a7e297e876d1e 

Title:    CHAPTER 26 Asthma, chronic obstructive<br>pulmonary disease, and pneumonia 

Abstract:   nan 

Abstract_Summary:   Not provided. 

> -------------------------------------------